### Quantize Keras Resnet50V1 Model

#### A data set that is a subset of Imagenet that has only 10 classes is used for training and validation 

A different training regime could yield different results from  that which is achieved in this file. Please install
tensorflow and tensorflow-model-optimization as shown below

pip install  tf-nightly

pip install tensorflow-model-optimization


The method used here has not been officially released and is experimental. It may
never make it to a release.  Sometimes I have run into problems where a particular release
does not work


In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.layers import ZeroPadding2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Softmax
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing import image
import tensorflow.keras.backend as K
from tensorflow.python.keras.utils import layer_utils
from tensorflow.python.keras.utils.data_utils import get_file
from tensorflow.python.keras.applications.imagenet_utils import decode_predictions
from tensorflow.python.keras.applications.imagenet_utils import preprocess_input
from tensorflow.python.keras.applications.imagenet_utils import obtain_input_shape
from tensorflow.keras.utils import get_source_inputs
#from roundInf import *
from tensorflow.keras import models
import matplotlib.pyplot as plt
from tensorflow.keras.initializers import glorot_uniform
import scipy.misc
from tensorflow.keras.utils import to_categorical
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_model_optimization as tfmot
from tensorflow.keras import regularizers


### Change numBits to select either 8 bit or 4 bit quantization

In 8 bit quantization the activations are 8 bit and the weights 5 bit
In 4 bit quantization both the activations and weights are 4 bit.  In 4 bit quantization changing the dense layer to 8 bit and possibly the input layer will yield better results.

In [2]:
numBits = 4
if(numBits ==8):
   import quant8
else:
   import quant4

In [3]:
batch_size=16
train_datagen = ImageDataGenerator(
                                     horizontal_flip=True,
                                     vertical_flip=False,
                                     rotation_range=20,
                                     preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
        'images/train/', 
        target_size=(224, 224), 
        batch_size=batch_size,
        shuffle=True,
        class_mode='categorical'
        )



Found 9469 images belonging to 10 classes.


In [1]:
validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
validation_generator = validation_datagen.flow_from_directory(
        'images/val/',  
        target_size=(224, 224),
        batch_size=batch_size,
        shuffle=True,
        class_mode='categorical'
        )

NameError: name 'ImageDataGenerator' is not defined

In [5]:
NUM_EPOCHS = 5
INIT_LR = 1e-4
class LossAndErrorPrintingCallback(tf.keras.callbacks.Callback):
    
    def on_epoch_end(self, epoch, logs=None):
      print('The average loss for epoch {} is {:7.2f} and the accuracy is {:7.2f}.'.format(epoch, logs['loss'], logs['accuracy']))
      print('The average validation loss for epoch {} is {:7.2f} and the validation accuracy is {:7.2f}.'.format(epoch, logs['val_loss'], logs['val_accuracy']))

In [6]:
opt1 =  tf.keras.optimizers.SGD(lr=INIT_LR,momentum=0.9, nesterov=True)

In [7]:
callbacks = [ LossAndErrorPrintingCallback()]

In [8]:
model = load_model('ModelResnet50V1.h5')
model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        zero_padding2d[0][0]             
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [9]:
quantize_annotate_layer = tfmot.quantization.keras.quantize_annotate_layer
quantize_annotate_model = tfmot.quantization.keras.quantize_annotate_model
quantize_scope = tfmot.quantization.keras.quantize_scope
if(numBits ==8):
    with quantize_scope(
      {'Conv2DQuantizeConfig':quant8.Conv2DQuantizeConfig,
      'ActivationQuantizeConfig': quant8.ActivationQuantizeConfig,
      'DenseQuantizeConfig': quant8.ActivationQuantizeConfig}):
 
       def apply_quantization(layer):
          if isinstance(layer, tf.keras.layers.Conv2D):
            return tfmot.quantization.keras.quantize_annotate_layer(layer,quant8.Conv2DQuantizeConfig())
          elif isinstance(layer, tf.keras.layers.Activation):
            return tfmot.quantization.keras.quantize_annotate_layer(layer,quant8.ActivationQuantizeConfig()) 
          elif isinstance(layer, tf.keras.layers.Dense):
            return tfmot.quantization.keras.quantize_annotate_layer(layer,quant8.DenseQuantizeConfig())  
          return layer
        
       annotated_model = tf.keras.models.clone_model(
       model,clone_function= apply_quantization)
       quant_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model)
       quant_aware_model.summary()
else:
    with quantize_scope(
      {'Conv2DQuantizeConfig':quant4.Conv2DQuantizeConfig,
      'ActivationQuantizeConfig': quant4.ActivationQuantizeConfig,
      'DenseQuantizeConfig': quant4.ActivationQuantizeConfig}):
 
       def apply_quantization(layer):
          if isinstance(layer, tf.keras.layers.Conv2D):
            return tfmot.quantization.keras.quantize_annotate_layer(layer,quant4.Conv2DQuantizeConfig())
          elif isinstance(layer, tf.keras.layers.Activation):
            return tfmot.quantization.keras.quantize_annotate_layer(layer,quant4.ActivationQuantizeConfig()) 
          elif isinstance(layer, tf.keras.layers.Dense):
            return tfmot.quantization.keras.quantize_annotate_layer(layer,quant4.DenseQuantizeConfig())  
          return layer
    
       annotated_model = tf.keras.models.clone_model(
       model,clone_function= apply_quantization)
       quant_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model)
       quant_aware_model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
quant_conv1 (QuantizeWrapper)   (None, 112, 112, 64) 9477        zero_padding2d[1][0]             
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         quant_conv1[0][0]                
___________________________________________________________________________________________

In [10]:
opt1 =  tf.keras.optimizers.SGD(lr=INIT_LR,momentum=0.9, nesterov=True)
quant_aware_model.compile(optimizer=opt1,
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [11]:
quant_aware_model.fit(train_generator,
                  batch_size=batch_size, steps_per_epoch=9469//batch_size, epochs=NUM_EPOCHS,verbose=0,
                  validation_data = validation_generator,
                  validation_steps=3929//batch_size,
                  callbacks = callbacks)

The average loss for epoch 0 is    2.33 and the accuracy is    0.63.
The average validation loss for epoch 0 is    2.34 and the validation accuracy is    0.65.
The average loss for epoch 1 is    2.14 and the accuracy is    0.70.
The average validation loss for epoch 1 is    2.28 and the validation accuracy is    0.66.
The average loss for epoch 2 is    2.09 and the accuracy is    0.70.
The average validation loss for epoch 2 is    2.14 and the validation accuracy is    0.70.
The average loss for epoch 3 is    2.05 and the accuracy is    0.71.
The average validation loss for epoch 3 is    2.23 and the validation accuracy is    0.67.
The average loss for epoch 4 is    2.02 and the accuracy is    0.71.
The average validation loss for epoch 4 is    2.08 and the validation accuracy is    0.70.


[NbConvertApp] Converting notebook Resnet50V1Quantization.ipynb to script
[NbConvertApp] Writing 7050 bytes to Resnet50V1Quantization.py
